# LLM Optimization Modelling Experiment

In [156]:
import vertexai
from vertexai.preview.generative_models import GenerativeModel
from IPython.display import Markdown

## 1. Define the problem description

In [563]:
problem = '''A buyer needs to acquire 239,600,480 units of a product and is considering bids from five suppliers, labeled A through E, each of whom can only supply a portion of the total required amount.
Each vendor has proposed different pricing structures, incorporating both setup fees and variable unit costs that change based on the quantity ordered.

The buyer's objective is to allocate the order among these suppliers to minimize overall costs, accounting for both setup and unit costs.

Vendor A offers a set up cost of $3855.34 and a unit cost of $61.150 per thousand of units.
Vendor A can supply up to 33 million units.

Vendor B offers a set up cost of $125,804.84 if purchasing between 22,000,000-70,000,000 units from vendor B with a unit cost of $68.099 per thousand units.
If purchasing between 70,000,001-100,000,000 units from vendor B, the set up cost increases to $269304.84 and the unit cost sinks to $66.049 per thousand units.
If purchasing between 100,000,001-150,000,000 units from vendor B, the unit cost per thousand units further decreases to $64.099, but the set up cost increases to $464304.84.
If purchasing between 150,000,001 and 160,000,000 units from vendor B, the unit cost is $62.119 per thousand units and the set up cost equals $761304.84.

Vendor C offers set up costs of $13,456.00 and a unit cost of $62.019 per thousand units.
Vendor C can supply up to 165.6 million units. Vendor D offers set up costs of $6,583.98 and a unit cost of $72.488 for a set of thousand units.

Vendor D can supply up to 12 million units at a price of $72.488 per thousand units and with a set up cost of $6583.98.

Vendor E offers free set up if purchasing between 0 and 42 million units of vendor E with a unit price of $70.150 per thousand units.
If purchasing between 42,000,001 and 77 million units from vendor E, the unit cost starts at $68.150 per thousand units, but with every one million units purchased the price decreases at a rate of 0.05 percent. An additional set up cost of $84000 will be charged as well.

Note that zero units may be purchased from vendor B: otherwise no positive number of units less than 22,000,000 may be purchased.'''

## 2. Generate the mathematical model

In [564]:
#Initializing the session. To replicate, make sure the right credentials are saved in a PATH variable
PROJECT_ID = "llm4optproblems"
REGION = "us-central1"
vertexai.init(project=PROJECT_ID, location=REGION)

#Specifying the model
generative_multimodal_model = GenerativeModel("gemini-1.5-pro-preview-0409")

#The propmt applied to all problems
prompt = '''Let's think step by step. Please write a mathematical optimization model for this problem. If there are parameter values, make sure to include them in the mathematical formulation.
'''

#Generate the response
response = generative_multimodal_model.generate_content([prompt+problem])


In [565]:
#Show the resopnse in a formatted way
Markdown(response.text)

## Mathematical Optimization Model

**Sets:**

*  $V = \{A, B, C, D, E\}$: Set of vendors.
*  $T_B = \{1, 2, 3, 4\}$: Set of pricing tiers for vendor B.

**Parameters:**

*  $R = 239,\!600,\!480$: Total units required.
*  $S_v$: Setup cost for vendor $v \in V$.
    *  $S_A = \$3,\!855.34$
    *  $S_{B,1} = \$125,\!804.84$
    *  $S_{B,2} = \$269,\!304.84$
    *  $S_{B,3} = \$464,\!304.84$
    *  $S_{B,4} = \$761,\!304.84$
    *  $S_C = \$13,\!456.00$
    *  $S_D = \$6,\!583.98$
    *  $S_{E,1} = \$0$
    *  $S_{E,2} = \$84,\!000$
*  $C_{v,t}$: Unit cost per thousand units for vendor $v \in V$ in pricing tier $t$. 
    *  $C_A = \$61.150$
    *  $C_{B,1} = \$68.099$
    *  $C_{B,2} = \$66.049$
    *  $C_{B,3} = \$64.099$
    *  $C_{B,4} = \$62.119$
    *  $C_C = \$62.019$
    *  $C_D = \$72.488$
    *  $C_{E,1} = \$70.150$
*  $L_{v,t}$: Lower bound on the quantity purchased from vendor $v \in V$ in pricing tier $t$.
    *  $L_{B,1} = 22,\!000,\!000$
    *  $L_{B,2} = 70,\!000,\!001$
    *  $L_{B,3} = 100,\!000,\!001$
    *  $L_{B,4} = 150,\!000,\!001$
    *  $L_{E,2} = 42,\!000,\!001$
*  $U_{v,t}$: Upper bound on the quantity purchased from vendor $v \in V$ in pricing tier $t$.
    *  $U_A = 33,\!000,\!000$
    *  $U_{B,1} = 70,\!000,\!000$
    *  $U_{B,2} = 100,\!000,\!000$
    *  $U_{B,3} = 150,\!000,\!000$
    *  $U_{B,4} = 160,\!000,\!000$
    *  $U_C = 165,\!600,\!000$
    *  $U_D = 12,\!000,\!000$
    *  $U_{E,1} = 42,\!000,\!000$
    *  $U_{E,2} = 77,\!000,\!000$
*  $\alpha = 0.05$: Price reduction rate per million units for vendor E in tier 2.

**Decision Variables:**

*  $x_v$: Quantity of units purchased from vendor $v \in V$.
*  $y_{v,t}$: Binary variable equal to 1 if purchasing from vendor $v \in V$ within pricing tier $t$, and 0 otherwise.

**Objective Function:**

Minimize the total cost:

```
Minimize:  ∑_{v∈V} (S_v * ∑_{t∈T_v} y_{v,t} + (C_v * x_v)/1000) 
```

**Constraints:**

1. **Demand Satisfaction:**
   ```
   ∑_{v∈V} x_v = R
   ```
2. **Vendor Capacity:**
   ```
   L_{v,t} * y_{v,t} ≤ x_v ≤ U_{v,t} * y_{v,t}, ∀ v ∈ V, t ∈ T_v
   ```
3. **Vendor B Tier Selection:**
   ```
   ∑_{t∈T_B} y_{B,t} ≤ 1
   ```
4. **Vendor E Pricing Tier 2 Discount:**
    * If $y_{E,2} = 1$, the unit price for vendor E decreases linearly with the quantity purchased: 
    ```
    C_E = C_{E,2} - α * (x_E - L_{E,2})/1,\!000,\!000
    ```
5. **Binary Variables:**
   ```
   y_{v,t} ∈ {0, 1}, ∀ v ∈ V, t ∈ T_v
   ```
6. **Non-negative Quantities:**
   ```
   x_v ≥ 0, ∀ v ∈ V
   ```

This model captures the problem's key aspects, including vendor-specific pricing tiers, setup costs, and capacity limits. The objective function minimizes the total cost, considering both setup and variable unit costs. The constraints ensure that demand is met, vendor capacities are respected, and only one pricing tier is selected for each vendor where applicable. 


## 3. Generate the pyomo code

In [566]:
#Second prompt gets the output of the previous step and generates the code
prompt2 = "Please write pyomo code for this mathematical problem. Use sample data where needed. Indicate where you use sample data. \n"
prompt2 += response.text 
response2 = generative_multimodal_model.generate_content([prompt2])

In [567]:
#Showing the code in a formatted way
Markdown(response2.text)

```python
import pyomo.environ as pyo

# Define sets
V = ['A', 'B', 'C', 'D', 'E']  # Vendors
T_B = [1, 2, 3, 4]  # Pricing tiers for vendor B
T_E = [1, 2]      # Pricing tiers for vendor E

# Define parameters
R = 239600480  # Total units required

S = {'A': 3855.34,
     'B': {1: 125804.84, 2: 269304.84, 3: 464304.84, 4: 761304.84},
     'C': 13456.00,
     'D': 6583.98,
     'E': {1: 0, 2: 84000}}  # Setup costs

C = {'A': 61.150,
     'B': {1: 68.099, 2: 66.049, 3: 64.099, 4: 62.119},
     'C': 62.019,
     'D': 72.488,
     'E': {1: 70.150, 2: 70.150}}  # Unit costs, note: C['E'][2] gets updated if tier 2 is chosen

L = {'B': {1: 22000000, 2: 70000001, 3: 100000001, 4: 150000001},
     'E': {2: 42000001}}  # Lower bounds

U = {'A': 33000000,
     'B': {1: 70000000, 2: 100000000, 3: 150000000, 4: 160000000},
     'C': 165600000,
     'D': 12000000,
     'E': {1: 42000000, 2: 77000000}}  # Upper bounds

alpha = 0.05  # Price reduction rate for vendor E in tier 2

# Create model
model = pyo.ConcreteModel()

# Define decision variables
model.x = pyo.Var(V, domain=pyo.NonNegativeReals)  # Quantity purchased from each vendor
model.y = pyo.Var(V, domain=pyo.Binary, dense=False)  # Binary variable for tier selection

# Define tier selection variables for vendors with tiers
model.y_B = pyo.Var(T_B, domain=pyo.Binary)
model.y_E = pyo.Var(T_E, domain=pyo.Binary)

# Define objective function
def objective_rule(model):
    total_setup_cost = sum(S[v] * sum(model.y[v] for t in T_v) for v, T_v in {'B': T_B, 'E': T_E}.items())
    total_variable_cost = sum((C[v] * model.x[v]) / 1000 for v in V)
    return total_setup_cost + total_variable_cost

model.objective = pyo.Objective(rule=objective_rule, sense=pyo.minimize)

# Define constraints

# Demand satisfaction
model.demand = pyo.Constraint(expr=sum(model.x[v] for v in V) == R)

# Vendor capacity
model.vendor_capacity = pyo.ConstraintList()
for v in V:
    if v == 'B':
        for t in T_B:
            model.vendor_capacity.add(L['B'][t] * model.y_B[t] <= model.x['B'] <= U['B'][t] * model.y_B[t])
    elif v == 'E':
        for t in T_E:
            model.vendor_capacity.add(L['E'][t] * model.y_E[t] <= model.x['E'] <= U['E'][t] * model.y_E[t])
    else:
        model.vendor_capacity.add(0 <= model.x[v] <= U[v])

# Vendor B tier selection
model.vendor_B_tier = pyo.Constraint(expr=sum(model.y_B[t] for t in T_B) <= 1)

# Vendor E pricing tier 2 discount
def vendor_E_discount_rule(model):
    if model.y_E[2] == 1:
        return C['E'][2] - alpha * (model.x['E'] - L['E'][2]) / 1000000 
    else:
        return C['E'][1] 

model.vendor_E_discount = pyo.Constraint(rule=vendor_E_discount_rule)

# Solve the model
solver = pyo.SolverFactory('gurobi')  # Choose a solver, in this case Gurobi
results = solver.solve(model)

# Print the results
print(results)

# Access the optimal solution
print("Optimal Solution:")
for v in V:
    print(f"Quantity from vendor {v}: {model.x[v].value}")
    if v == 'B':
        for t in T_B:
            print(f"Vendor B Tier {t}: {model.y_B[t].value}")
    if v == 'E':
        for t in T_E:
            print(f"Vendor E Tier {t}: {model.y_E[t].value}")

print(f"Total cost: {model.objective.expr()}")
```

**Sample Data Used:**

The following parameters were populated with sample data based on the provided information in the problem statement. 

*   `R`: Total units required 
*   `S`: Setup cost for vendor $v ∈ V$
*   `C`: Unit cost per thousand units for vendor  $v ∈ V$ in pricing tier $t$
*   `L`: Lower bound on the quantity purchased from vendor $v ∈ V$ in pricing tier $t$
*   `U`: Upper bound on the quantity purchased from vendor $v ∈ V$ in pricing tier $t$
*   `alpha`: Price reduction rate per million units for vendor E in tier 2.

**Note:** 

*   This code uses the Gurobi solver. You need to have Gurobi installed and a valid license to run this code. 
*   This code defines sets for the pricing tiers of vendors B and E (`T_B` and `T_E`).
*   The discount for Vendor E is implemented using a constraint `model.vendor_E_discount`.
*   The variable `model.y` represents the general choice of a vendor, regardless of the tier. For vendors B and E, there are additional variables `model.y_B` and `model.y_E` to represent the choice of tier.  The capacity constraints ensure that the quantities are consistent with the selected tier. 


## 4. Input problem data and try running the generated code

In [568]:
import pyomo.environ as pyo

# Define sets
V = ['A', 'B', 'C', 'D', 'E']  # Vendors
T_B = [1, 2, 3, 4]  # Pricing tiers for vendor B
T_E = [1, 2]      # Pricing tiers for vendor E

# Define parameters
R = 239600480  # Total units required

S = {'A': 3855.34,
     'B': {1: 125804.84, 2: 269304.84, 3: 464304.84, 4: 761304.84},
     'C': 13456.00,
     'D': 6583.98,
     'E': {1: 0, 2: 84000}}  # Setup costs

C = {'A': 61.150,
     'B': {1: 68.099, 2: 66.049, 3: 64.099, 4: 62.119},
     'C': 62.019,
     'D': 72.488,
     'E': {1: 70.150, 2: 70.150}}  # Unit costs, note: C['E'][2] gets updated if tier 2 is chosen

L = {'B': {1: 22000000, 2: 70000001, 3: 100000001, 4: 150000001},
     'E': {2: 42000001}}  # Lower bounds

U = {'A': 33000000,
     'B': {1: 70000000, 2: 100000000, 3: 150000000, 4: 160000000},
     'C': 165600000,
     'D': 12000000,
     'E': {1: 42000000, 2: 77000000}}  # Upper bounds

alpha = 0.05  # Price reduction rate for vendor E in tier 2

# Create model
model = pyo.ConcreteModel()

# Define decision variables
model.x = pyo.Var(V, domain=pyo.NonNegativeReals)  # Quantity purchased from each vendor
model.y = pyo.Var(V, domain=pyo.Binary, dense=False)  # Binary variable for tier selection

# Define tier selection variables for vendors with tiers
model.y_B = pyo.Var(T_B, domain=pyo.Binary)
model.y_E = pyo.Var(T_E, domain=pyo.Binary)

# Define objective function
def objective_rule(model):
    total_setup_cost = sum(S[v] * sum(model.y[v] for t in T_v) for v, T_v in {'B': T_B, 'E': T_E}.items())
    total_variable_cost = sum((C[v] * model.x[v]) / 1000 for v in V)
    return total_setup_cost + total_variable_cost

model.objective = pyo.Objective(rule=objective_rule, sense=pyo.minimize)

# Define constraints

# Demand satisfaction
model.demand = pyo.Constraint(expr=sum(model.x[v] for v in V) == R)

# Vendor capacity
model.vendor_capacity = pyo.ConstraintList()
for v in V:
    if v == 'B':
        for t in T_B:
            model.vendor_capacity.add(L['B'][t] * model.y_B[t] <= model.x['B'] <= U['B'][t] * model.y_B[t])
    elif v == 'E':
        for t in T_E:
            model.vendor_capacity.add(L['E'][t] * model.y_E[t] <= model.x['E'] <= U['E'][t] * model.y_E[t])
    else:
        model.vendor_capacity.add(0 <= model.x[v] <= U[v])

# Vendor B tier selection
model.vendor_B_tier = pyo.Constraint(expr=sum(model.y_B[t] for t in T_B) <= 1)

# Vendor E pricing tier 2 discount
def vendor_E_discount_rule(model):
    if model.y_E[2] == 1:
        return C['E'][2] - alpha * (model.x['E'] - L['E'][2]) / 1000000 
    else:
        return C['E'][1] 

model.vendor_E_discount = pyo.Constraint(rule=vendor_E_discount_rule)

# Solve the model
solver = pyo.SolverFactory('gurobi')  # Choose a solver, in this case Gurobi
results = solver.solve(model)

# Print the results
print(results)

# Access the optimal solution
print("Optimal Solution:")
for v in V:
    print(f"Quantity from vendor {v}: {model.x[v].value}")
    if v == 'B':
        for t in T_B:
            print(f"Vendor B Tier {t}: {model.y_B[t].value}")
    if v == 'E':
        for t in T_E:
            print(f"Vendor E Tier {t}: {model.y_E[t].value}")

print(f"Total cost: {model.objective.expr()}")

ERROR: Rule failed when generating expression for Objective objective with
index None: TypeError: unsupported operand type(s) for *: 'dict' and
'LinearExpression'
ERROR: Constructing component 'objective' from data=None failed:
        TypeError: unsupported operand type(s) for *: 'dict' and
        'LinearExpression'


TypeError: unsupported operand type(s) for *: 'dict' and 'LinearExpression'

## 5. Correct the code to verify model viability (optional)

## 6. Printing the outputs as strings, so they can be saved.
Those can be rendered as markdown for better readability

In [569]:
print(response.text)

## Mathematical Optimization Model

**Sets:**

*  $V = \{A, B, C, D, E\}$: Set of vendors.
*  $T_B = \{1, 2, 3, 4\}$: Set of pricing tiers for vendor B.

**Parameters:**

*  $R = 239,\!600,\!480$: Total units required.
*  $S_v$: Setup cost for vendor $v \in V$.
    *  $S_A = \$3,\!855.34$
    *  $S_{B,1} = \$125,\!804.84$
    *  $S_{B,2} = \$269,\!304.84$
    *  $S_{B,3} = \$464,\!304.84$
    *  $S_{B,4} = \$761,\!304.84$
    *  $S_C = \$13,\!456.00$
    *  $S_D = \$6,\!583.98$
    *  $S_{E,1} = \$0$
    *  $S_{E,2} = \$84,\!000$
*  $C_{v,t}$: Unit cost per thousand units for vendor $v \in V$ in pricing tier $t$. 
    *  $C_A = \$61.150$
    *  $C_{B,1} = \$68.099$
    *  $C_{B,2} = \$66.049$
    *  $C_{B,3} = \$64.099$
    *  $C_{B,4} = \$62.119$
    *  $C_C = \$62.019$
    *  $C_D = \$72.488$
    *  $C_{E,1} = \$70.150$
*  $L_{v,t}$: Lower bound on the quantity purchased from vendor $v \in V$ in pricing tier $t$.
    *  $L_{B,1} = 22,\!000,\!000$
    *  $L_{B,2} = 70,\!000,\!001$
   

In [571]:
print(response2.text)

```python
import pyomo.environ as pyo

# Define sets
V = ['A', 'B', 'C', 'D', 'E']  # Vendors
T_B = [1, 2, 3, 4]  # Pricing tiers for vendor B
T_E = [1, 2]      # Pricing tiers for vendor E

# Define parameters
R = 239600480  # Total units required

S = {'A': 3855.34,
     'B': {1: 125804.84, 2: 269304.84, 3: 464304.84, 4: 761304.84},
     'C': 13456.00,
     'D': 6583.98,
     'E': {1: 0, 2: 84000}}  # Setup costs

C = {'A': 61.150,
     'B': {1: 68.099, 2: 66.049, 3: 64.099, 4: 62.119},
     'C': 62.019,
     'D': 72.488,
     'E': {1: 70.150, 2: 70.150}}  # Unit costs, note: C['E'][2] gets updated if tier 2 is chosen

L = {'B': {1: 22000000, 2: 70000001, 3: 100000001, 4: 150000001},
     'E': {2: 42000001}}  # Lower bounds

U = {'A': 33000000,
     'B': {1: 70000000, 2: 100000000, 3: 150000000, 4: 160000000},
     'C': 165600000,
     'D': 12000000,
     'E': {1: 42000000, 2: 77000000}}  # Upper bounds

alpha = 0.05  # Price reduction rate for vendor E in tier 2

# Create model
mode